# SAE Project: Industry 4.0 Quality Control with OpenManipulator-X

## Project Overview

**Context:** You are a robotics engineer at a smart factory implementing Industry 4.0 solutions. Your company manufactures electronic components and needs to automate the quality inspection and sorting process.

**Mission:** Design and implement a robotic cell using the OpenManipulator-X manipulator for automated quality inspection and sorting of electronic components.

---

## Learning Objectives

By completing this project, you will:
1. Apply forward kinematics (DH, DH-modified, elementary transformations) to a real robot
2. Use inverse kinematics to plan robot motions
3. Implement Jacobian-based velocity control
4. Design smooth trajectories for pick-and-place operations
5. Integrate sensor data for quality inspection
6. Develop a complete Industry 4.0 robotic application

---

## Project Duration

- **Total Time:** 52 hours (7 sessions)
- **Session 1:** Project Introduction and Setup (4h)
- **Session 2:** Robot modeling and workspace analysis (8h)
- **Session 3:** Vision System and Object Localization (8h)
- **Session 4:** System Integration (8h)
- **Session 5:** Advanced Features and Optimization (8h)
- **Session 6:** Testing, Validation, and Documentation (8h)
- **Session 7:** Final Integration and Presentation (8h)

---

## The Challenge: Smart Quality Control Cell

### Scenario Description

Your factory produces small electronic components (PCBs, connectors, sensors) on a conveyor belt. These components need to be:

1. **Picked** from an incoming tray (simulated conveyor position)
2. **Inspected** at a vision system station
3. **Sorted** into three categories:
   - **GOOD**: Meets all quality standards → Green bin
   - **REWORK**: Minor defects, can be repaired → Yellow bin
   - **REJECT**: Major defects → Red bin

### Workspace Layout

```
                    Top View
                    
        [Green Bin]     [Yellow Bin]     [Red Bin]
            (-150,200)      (0,220)        (150,200)
            
                    [Vision Station]
                        (0, 150)
                        
                    [OpenManipulator-X]
                         (0, 0)
                         
        [Incoming Tray with 3x3 grid of components]
                    (0, -200 to -150)
```

### Technical Specifications

- **Robot:** OpenManipulator-X (4-DOF manipulator)
- **Components size:** 20mm × 20mm × 10mm
- **Bin positions:** Must be within robot workspace
- **Cycle time target:** < 8 seconds per component
- **Quality criteria:** Simulated sensor data (position, orientation, defect type)

---

## Deliverables

### 1. Technical Report (60%)

Your report must include:

#### A. Robot Modeling (15%)
- Complete kinematic model using DH parameters
- Verification using DH-modified convention
- Forward kinematics validation with robotics-toolbox-python
- Workspace analysis and reachability study

#### B. Motion Planning (20%)
- Inverse kinematics solutions for key positions
- Joint-space trajectory generation (pick-place-sort)
- Cartesian-space trajectory for inspection scanning
- Velocity and acceleration profiles

#### C. Quality Inspection Algorithm (15%)
- Sensor integration strategy
- Decision logic implementation
- Real-time classification algorithm

#### D. System Integration (10%)
- Complete workflow implementation
- Cycle time optimization
- Error handling and safety considerations

### 2. Python Implementation (30%)

- Well-documented code with clear structure
- Use of robotics-toolbox-python for simulation
- Visualization of robot motion and workspace
- Performance metrics and analysis

### 3. Demonstration (10%)

- Live simulation demonstration
- Q&A session on design choices
- Presentation of results and challenges encountered

---

## Getting Started

### Required Libraries

In [14]:
# Install required packages (run once)
# !pip install roboticstoolbox-python numpy matplotlib scipy

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from roboticstoolbox import DHRobot, RevoluteDH, PrismaticDH
from spatialmath import SE3
import roboticstoolbox as rtb
from scipy.interpolate import CubicSpline
from mpl_toolkits.mplot3d import Axes3D

# Set plotting style
plt.style.use('seaborn-darkgrid')
%matplotlib inline

---

# SESSION 1: Robot Modeling and Workspace Analysis (4 hours)

## Objectives
- Model the OpenManipulator-X robot using DH parameters
- Implement forward kinematics
- Analyze workspace and verify bin positions are reachable

---

## Part 1.1: OpenManipulator-X Specifications

The OpenManipulator-X is a 4-DOF robotic arm with the following characteristics:

### Link Lengths (mm)
- Base height: 77 mm
- Link 1 (shoulder to elbow): 130 mm
- Link 2 (elbow to wrist): 124 mm
- Link 3 (wrist to gripper): 126 mm

### Joint Limits (degrees)
- Joint 1 (base rotation): -180° to +180°
- Joint 2 (shoulder): -90° to +90°
- Joint 3 (elbow): -80° to +95°
- Joint 4 (wrist): -90° to +90°

### Coordinate Frame Assignment

```
      
          ↑   
      ┌───┼───┐  Gripper
      │   │   │  
      └───┴───┘  
          |
          | Link 3 (126mm)
          |
       Z₃ ↑
          ●───→ X₃  (wrist joint)
          │
          │ Link 2 (124mm)
          │
       Z₂ ↑
          ●───→ X₂  (Elbow joint)
          │
          │ Link 1 (130mm)
          │
       Z₁ ↑
          ●───→ X₁  (Shoulder joint)
          │
          │ 77mm
          │
       Z₀ ↑
       ───●───→ X₀  (Base)
```

---

## Task 1.1: DH Parameter Table (Standard Convention)

**Instructions:**
1. Complete the DH parameter table below
2. Use the standard DH convention (Craig)
3. Verify your parameters by sketching the frames

| Joint i | θᵢ (deg) | dᵢ (mm) | aᵢ (mm) | αᵢ (deg) |
|---------|----------|---------|---------|----------|
| 1       | θ₁*      | ?       | ?       | ?        |
| 2       | θ₂*      | ?       | ?       | ?        |
| 3       | θ₃*      | ?       | ?       | ?        |
| 4       | θ₄*      | ?       | ?       | ?        |

*θᵢ = variable joint angle

**✍️ Hand-written solution required in your report**

In [16]:
# Task 1.1: Implement the OpenManipulator-X using standard DH parameters

class OpenManipulatorX_DH(DHRobot):
    """
    OpenManipulator-X robot model using standard DH convention
    """
    def __init__(self):
        # TODO: Complete the DH parameters
        # Link dimensions in meters
        d1 = 0.077  # Base height
        a2 = 0.130  # Link 1 length
        a3 = 0.124  # Link 2 length
        a4 = 0.126  # Link 3 length
        
        # Define links using RevoluteDH(d, a, alpha, offset)
        L = [
            RevoluteDH(d=d1, a=0, alpha=np.pi/2, offset=0),  # Joint 1
            # TODO: Add joints 2, 3, 4
        ]
        
        super().__init__(
            L,
            name="OpenManipulator-X (DH)",
            manufacturer="ROBOTIS"
        )
        
        # Define joint limits (in radians)
        self.qlim = np.array([
            [-np.pi, np.pi],
            [-np.pi/2, np.pi/2],
            [-80*np.pi/180, 95*np.pi/180],
            [-np.pi/2, np.pi/2]
        ])

# Create robot instance
# robot_dh = OpenManipulatorX_DH()
# print(robot_dh)

## Task 1.2: Modified DH Parameters

**Instructions:**
1. Convert your standard DH parameters to modified DH convention (Khalil-Kleinfinger)
2. Complete the table below
3. Explain the differences between the two conventions

| Joint i | αᵢ₋₁ (deg) | aᵢ₋₁ (mm) | dᵢ (mm) | θᵢ (deg) |
|---------|------------|-----------|---------|----------|
| 1       | ?          | ?         | ?       | θ₁*      |
| 2       | ?          | ?         | ?       | θ₂*      |
| 3       | ?          | ?         | ?       | θ₃*      |
| 4       | ?          | ?         | ?       | θ₄*      |

**✍️ Hand-written solution required in your report**

In [17]:
# Task 1.2: Implement using Modified DH parameters

from roboticstoolbox import RevoluteMDH

class OpenManipulatorX_MDH(DHRobot):
    """
    OpenManipulator-X robot model using modified DH convention
    """
    def __init__(self):
        # TODO: Complete the modified DH parameters
        d1 = 0.077
        a2 = 0.130
        a3 = 0.124
        a4 = 0.126
        
        # Define links using RevoluteMDH(alpha, a, d, offset)
        L = [
            # TODO: Complete modified DH links
        ]
        
        super().__init__(
            L,
            name="OpenManipulator-X (Modified DH)",
            manufacturer="ROBOTIS"
        )

# robot_mdh = OpenManipulatorX_MDH()
# print(robot_mdh)

## Task 1.3: Elementary Transformation Method

**Instructions:**
1. Express the forward kinematics using elementary transformations
2. Write the complete transformation matrix T₀₄
3. Compare the results with DH method

The transformation from base to end-effector can be written as:

$$T_0^4 = Trans(Z_0, d_1) \cdot Rot(Z_0, \theta_1) \cdot Trans(X_1, a_1) \cdot ... $$

**✍️ Hand-written derivation required in your report**

In [18]:
# Task 1.3: Implement forward kinematics using elementary transformations

def fk_elementary_transforms(q):
    """
    Forward kinematics using elementary transformations
    
    Args:
        q: Joint angles [q1, q2, q3, q4] in radians
    
    Returns:
        T: 4x4 transformation matrix from base to end-effector
    """
    q1, q2, q3, q4 = q
    
    # Robot dimensions
    d1 = 0.077
    a2 = 0.130
    a3 = 0.124
    a4 = 0.126
    
    # TODO: Build transformation using elementary transforms
    # Example: T = SE3.Tz(d1) @ SE3.Rz(q1) @ ...
    
    T = SE3()  # Placeholder
    
    return T

# Test the function
# q_test = [0, 0, 0, 0]  # Home position
# T_elem = fk_elementary_transforms(q_test)
# print("Elementary transforms result:")
# print(T_elem)

## Task 1.4: Forward Kinematics Validation

Verify that all three methods (DH, Modified DH, Elementary) give the same results.

In [19]:
# Task 1.4: Compare the three methods

def compare_fk_methods(q):
    """
    Compare forward kinematics from three different methods
    """
    print(f"Joint configuration: {np.rad2deg(q)} degrees\n")
    
    # Method 1: Standard DH
    # T_dh = robot_dh.fkine(q)
    # print("Standard DH:")
    # print(T_dh)
    
    # Method 2: Modified DH
    # T_mdh = robot_mdh.fkine(q)
    # print("\nModified DH:")
    # print(T_mdh)
    
    # Method 3: Elementary transforms
    # T_elem = fk_elementary_transforms(q)
    # print("\nElementary Transforms:")
    # print(T_elem)
    
    # Check if results are equal (within tolerance)
    # TODO: Compare transformation matrices
    
    pass

# Test configurations
test_configs = [
    [0, 0, 0, 0],  # Home
    [np.pi/4, -np.pi/4, np.pi/4, 0],  # Arbitrary pose
    [0, np.pi/3, -np.pi/3, np.pi/6],  # Another pose
]

# for q in test_configs:
#     compare_fk_methods(q)
#     print("="*50)

## Task 1.5: Workspace Analysis

**Objectives:**
1. Analyze the robot's reachable workspace
2. Verify that all bin positions are within the workspace
3. Generate workspace visualization (top view and side view)

In [20]:
# Task 1.5: Workspace analysis

def compute_workspace_points(robot, n_samples=10000):
    """
    Generate random configurations and compute end-effector positions
    
    Args:
        robot: Robot model
        n_samples: Number of random configurations to test
    
    Returns:
        points: Nx3 array of end-effector positions
    """
    points = []
    
    for _ in range(n_samples):
        # Generate random joint configuration within limits
        # q = robot.random_q()
        # T = robot.fkine(q)
        # points.append(T.t)  # Extract position
        pass
    
    return np.array(points)

# Generate workspace
# workspace_points = compute_workspace_points(robot_dh)

# print(f"Workspace bounds:")
# print(f"X: [{workspace_points[:, 0].min():.3f}, {workspace_points[:, 0].max():.3f}] m")
# print(f"Y: [{workspace_points[:, 1].min():.3f}, {workspace_points[:, 1].max():.3f}] m")
# print(f"Z: [{workspace_points[:, 2].min():.3f}, {workspace_points[:, 2].max():.3f}] m")

In [21]:
# Visualize workspace with bin positions

def plot_workspace_with_bins(workspace_points):
    """
    Plot workspace and verify bin positions are reachable
    """
    # Define bin positions (in mm, convert to m)
    bin_positions = {
        'Green (GOOD)': np.array([-150, 200, 100]) / 1000,
        'Yellow (REWORK)': np.array([0, 220, 100]) / 1000,
        'Red (REJECT)': np.array([150, 200, 100]) / 1000,
        'Vision Station': np.array([0, 150, 80]) / 1000,
        'Incoming Tray': np.array([0, -180, 50]) / 1000,
    }
    
    fig = plt.figure(figsize=(15, 5))
    
    # Top view (X-Y plane)
    ax1 = fig.add_subplot(131)
    # ax1.scatter(workspace_points[:, 0], workspace_points[:, 1], 
    #            s=0.1, alpha=0.3, c='blue', label='Workspace')
    
    # Plot bin positions
    # for name, pos in bin_positions.items():
    #     ax1.scatter(pos[0], pos[1], s=100, marker='x', label=name)
    
    ax1.set_xlabel('X (m)')
    ax1.set_ylabel('Y (m)')
    ax1.set_title('Top View (X-Y)')
    ax1.legend()
    ax1.grid(True)
    ax1.axis('equal')
    
    # Side view (X-Z plane)
    ax2 = fig.add_subplot(132)
    # ax2.scatter(workspace_points[:, 0], workspace_points[:, 2], 
    #            s=0.1, alpha=0.3, c='blue')
    # for name, pos in bin_positions.items():
    #     ax2.scatter(pos[0], pos[2], s=100, marker='x')
    
    ax2.set_xlabel('X (m)')
    ax2.set_ylabel('Z (m)')
    ax2.set_title('Side View (X-Z)')
    ax2.grid(True)
    ax2.axis('equal')
    
    # Front view (Y-Z plane)
    ax3 = fig.add_subplot(133)
    # ax3.scatter(workspace_points[:, 1], workspace_points[:, 2], 
    #            s=0.1, alpha=0.3, c='blue')
    # for name, pos in bin_positions.items():
    #     ax3.scatter(pos[1], pos[2], s=100, marker='x')
    
    ax3.set_xlabel('Y (m)')
    ax3.set_ylabel('Z (m)')
    ax3.set_title('Front View (Y-Z)')
    ax3.grid(True)
    ax3.axis('equal')
    
    plt.tight_layout()
    plt.show()

# plot_workspace_with_bins(workspace_points)

## Task 1.6: Reachability Check

**Instructions:**
1. Implement a function to check if a target position is within the workspace
2. Verify all bin positions are reachable
3. If any position is unreachable, propose adjustments to the workspace layout

In [22]:
# Task 1.6: Check reachability of key positions

def check_position_reachable(robot, target_pos, target_orient=None, n_attempts=10):
    """
    Check if a target position is reachable by the robot
    
    Args:
        robot: Robot model
        target_pos: [x, y, z] target position in meters
        target_orient: Optional orientation (RPY or rotation matrix)
        n_attempts: Number of IK attempts with different initial guesses
    
    Returns:
        reachable: Boolean
        q_solution: Joint configuration if reachable, None otherwise
    """
    # TODO: Implement reachability check
    # Hint: Use robot.ikine_LM() or robot.ikine_GN()
    
    return False, None

# Check all key positions
key_positions = {
    'Green Bin': [-0.150, 0.200, 0.100],
    'Yellow Bin': [0.000, 0.220, 0.100],
    'Red Bin': [0.150, 0.200, 0.100],
    'Vision Station': [0.000, 0.150, 0.080],
    'Tray Center': [0.000, -0.180, 0.050],
}

# for name, pos in key_positions.items():
#     reachable, q = check_position_reachable(robot_dh, pos)
#     status = "✓ REACHABLE" if reachable else "✗ NOT REACHABLE"
#     print(f"{name:20s}: {status}")
#     if reachable:
#         print(f"  Joint config: {np.rad2deg(q)}")

---

## Session 1 Deliverables

**By the end of Session 1, you should have:**

✅ Complete DH parameter table (standard and modified conventions)  
✅ Elementary transformation derivation  
✅ Working forward kinematics implementation (all three methods)  
✅ Workspace analysis with visualization  
✅ Verification that all target positions are reachable  
✅ Documented comparison of the three kinematic modeling approaches  

**Questions for Report:**

1. What are the advantages and disadvantages of each kinematic modeling method?
2. How does the workspace shape reflect the robot's joint limits?
3. What is the maximum reach of the OpenManipulator-X?
4. Are there any singularities within the workspace? If so, where?
5. Based on your workspace analysis, would you recommend any changes to the bin positions?

---

**Continue to Session 2 notebook for motion planning and trajectory generation...**